# Métodos no supervisados: Topic Modeling y Clustering
Cuándo se está trabajando con un dataset compuesto de una gran cantidad de documentos se quiere saber cuál es el tema del que tratan. Es muy complicado asignar un único tema al corpus debido a qué está compuesto por textos con gran variedad de temas, más en este caso en el que se trabaja con un dataset conformado por comentarios de usuarios. 

Es en este punto cuando entra en juego el Topic Modeling (Modelado de temas). Este tipo de modelado se utiliza para determinar la estructura global del corpus mediante diversas técnicas estadísticas, no para asignar un tema concreto a cada uno de los documentos del corpus.

En este apartado se estudiarán varios métodos para el modelado de temas que permitirán entender su funcionamiento y cómo pueden ser utilizados para generar resúmenes de los documentos de forma rápida.

Para esta tarea se utilizará el dataset conformado por los comentarios de usuarios del repositorio "Zigbee2mqtt" que ya ha sido utilizado en tareas anteriores, lo que permite agilizar el proceso de modelado de temas porque este ya está tokenizado, vectorizado, etc.

Al igual que en todos los apartados, se comienza importando los ajustes del proyecto junto con la base de datos en la que se encuentra almacenado el Data Frame.

In [1]:
import sys, os

#Carga del archivo setup.py
%run -i ../pyenv_settings/setup.py

#Imports y configuraciones de gráficas
%run "$BASE_DIR/pyenv_settings/settings.py"

%reload_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'png'

# # to print output of all statements and not just the last
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# # otherwise text between $ signs will be interpreted as formula and printed in italic
pd.set_option('display.html.use_mathjax', False)

You are working on a local system.
Files will be searched relative to "..".


In [2]:
#Conexión con la base de datos en la que tenemos guardado el Data Frame
db_name = "../data/zigbee2mqtt_comments.db"
con = sqlite3.connect(db_name)
df = pd.read_sql("select * from comments", con)
con.close()

#Comprobación de que se ha cargado correctamente
print(df.columns)
print(df[['normalized_text', 'tokens']].head(4))

Index(['id', 'user', 'text', 'impurity', 'clean_text', 'normalized_text',
       'tokens'],
      dtype='object')
                                                                                                                                                                                           normalized_text  \
0                                                                    This issue is stale because it has been open 30 days with no activity. Remove stale label or comment or this will be closed in 7 days   
1  Also, after updating the z2m, cyclic reboots began ''' Starting Zigbee2MQTT without watchdog. INFO: Preparing to start... INFO: Socat not enabled INFO: Starting Zigbee2MQTT... Starting Zigbee2MQTT...   
2  Hi ! Since 2 or 3 days, MQTT suddenly fail. A few messages in the log, many auto restart, and works again ... Very strange. In the log INFO: Preparing to start... ERROR: Got unexpected response fr...   
3  I don't know if it's exactly the same, but since v1.42 I ha

**Recordar que en la columna "normalized_text" se encuentran todos los comentarios ya normalizados y vectorizados, es decir, con todos los pasos que se han seguido para obtener un texto limpio y listo para utilizarse en tareas de modelado**

## Pasos previos
Antes de comenzar el modelado, es recomendable conocer la información del corpues para así determinar cuáles son las entidades que se analizarán.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2678 entries, 0 to 2677
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               2678 non-null   int64  
 1   user             2678 non-null   object 
 2   text             2678 non-null   object 
 3   impurity         2678 non-null   float64
 4   clean_text       2678 non-null   object 
 5   normalized_text  2678 non-null   object 
 6   tokens           2678 non-null   object 
dtypes: float64(1), int64(1), object(5)
memory usage: 146.6+ KB


A priori parece una buena base para el modelado teniendo en cuenta que no hay elemenos nulos en ninguna columna. De todos modos, se puede imprimir por pantalla alguna muestra de estos tectos para comprobar si estos contienen caracteres especiales como puede ser la tabulación (\t), nueva línea (\r), retorno de carro (\r), etc.

In [4]:
print(repr(df.iloc[0]["normalized_text"][0:200]))
print(repr(df.iloc[-1]["normalized_text"][0:200]))

'This issue is stale because it has been open 30 days with no activity. Remove stale label or comment or this will be closed in 7 days'
"Is thist error? I have (old config) ''' socat: restartdelay: 1 initialdelay: 1 ''' in config but when i check default config this positions will disaapear."


Como el data frame ya había sido procesado y limpiado previamente, no se encuentran caracteres especiales que puedan dificultar el modelado.

Una vez completado este paso, se pasaría a la vectorización de los datos. En este caso, los datos ya fueron vectorizados anteriormente en *e_featureEngineering_and_syntactSimilarity.ipynb* de este mismo proyecto, permitiendo ahorrar tiempo y agilizar el proceso para cumplir el objetivo de este apartado.

## Factorización de matrices no negativas (NMF)
Conceptualmente, la forma más sencilla de hallar la estructura implícita en el corpus es la factorización de la matriz de términos, pero puede presentar un gasto computacional muy elevado. 

En lugar de esto, se puede realizar una factorización aproximada que es menos costoso y al mismo tiempo arroja buenos resultados.

Algunos métodos de álgebra lineal permiten representar la matriz como el producto de otras dos matrices no negativas. En este caso se nombrará la matriz original como *V*, y los factores *W* y *H*. 

### Creación de Modelos temáticos utilizando NMF
Casi todos los modelados de temas necesitan un número de temas como parámetro de entrada. En lugar de utilizar todos los temas de todos los textos, se utilizará un número aleatorio para esta tarea, por ejemplo 10. Este número es variable en cualquier caso, al fin y al cabo lo que se busca es un resultado más afinado, pero tampoco puede ser un número demasiado elevado de forma que el gasto computacional exceda los valores deseados.

Como la vectorización de los textos fue llevada a cabo en otro documento del proyecto, en este se deberá realizar el proceso otra vez para poder hacer uso de la variable que almacena los vectores.

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from spacy.lang.en.stop_words import STOP_WORDS as stopwords

tfidf_text_vectorizer = TfidfVectorizer(stop_words=list(stopwords), min_df=5, max_df=0.7)
tfidf_text_vectors = tfidf_text_vectorizer.fit_transform(df['normalized_text'])
tfidf_text_vectors.shape

(2678, 1721)

In [6]:
# Comprobación de que no se pierde información
df['normalized_text'].info()
print(df['normalized_text'].sample(5))

<class 'pandas.core.series.Series'>
RangeIndex: 2678 entries, 0 to 2677
Series name: normalized_text
Non-Null Count  Dtype 
--------------  ----- 
2678 non-null   object
dtypes: object(1)
memory usage: 21.0+ KB
524                                                                                                                                                                    Please provide your feedback here: _URL_
1512                               Hi, sorry i forget to answer. Thanks for the tip, it's great. Should be nice to have version in an attribute somewhere, but it's egde, not supposed to follow every update..
374     > adapter: ember didn't work for me z2m: Error: NCP EZSP protocol version of 8 does not match Host version 13 at EmberAdapter.emberVersion (/app/node_modules/zigbee-herdsman/src/adapter/ember/adap...
435                                                                                       Which version of Home Assistant are you using? _URL_ says at least version 

In [7]:
from sklearn.decomposition import NMF

nmf_text_model = NMF(n_components=10, random_state=42)
W_text_matrix = nmf_text_model.fit_transform(tfidf_text_vectors)
H_text_matrix = nmf_text_model.components_

El tema de un texto viene dado por la distribución de las palabras que contiene, por lo tanto, analizar esta distribución puede ser de gran ayuda para descubrir los temas.

Haciendo uso de la matriz H, se debe encontrar el índice de los mayores valores de cada fila para luego ser utilizado como índice de búsqueda en el vocabulario.

Se definirá una función que evalúe el modelo y muestre por pantalla un resumen de los temas (topics) que NMF ha detectado en los textos.

In [8]:
def display_topics(model, features, no_top_words=5):
    for topic, words in enumerate(model.components_):
        total = words.sum()
        largest = words.argsort()[::-1] # invert sort order
        print("\nTopic %02d" % topic)
        for i in range(0, no_top_words):
            print("  %s (%2.2f)" % (features[largest[i]], abs(words[largest[i]]*100.0/total)))

In [9]:
display_topics(nmf_text_model, tfidf_text_vectorizer.get_feature_names_out())


Topic 00
  stale (18.06)
  days (17.31)
  activity (8.96)
  label (8.96)
  comment (8.86)

Topic 01
  add (2.32)
  z2m (2.01)
  version (1.86)
  ha (1.43)
  new (1.23)

Topic 02
  thanks (50.82)
  worked (3.38)
  working (1.87)
  lot (1.09)
  reply (0.99)

Topic 03
  issue (41.26)
  having (2.15)
  got (1.62)
  close (1.11)
  exact (1.07)

Topic 04
  _url_ (29.25)
  duplicate (4.55)
  related (2.59)
  closing (1.28)
  device (1.22)

Topic 05
  error (3.03)
  zigbee (2.55)
  zigbee2mqtt (2.52)
  herdsman (2.11)
  info (2.04)

Topic 06
  problem (21.86)
  solution (3.05)
  solved (2.78)
  zha (1.10)
  thank (1.09)

Topic 07
  config (4.15)
  zigbee2mqtt (2.28)
  configuration (2.24)
  yaml (2.03)
  addon (1.85)

Topic 08
  fixed (16.32)
  edge (5.29)
  dev (4.58)
  branch (4.31)
  latest (3.39)

Topic 09
  update (13.43)
  42 (3.94)
  working (1.91)
  version (1.40)
  updated (1.36)


La salida muestra las palabras más relevantes para cada tema, permitiendo determinar la temática de algunos de los comentarios. Por ejemplo, en *Topic 02* se puede deducir que se trata de un mensaje de agradeciemiento como respuesta a otro y en *Topic 04* que se trata de algún problema con una URL concreta.

Sería interesante conocer como de grande es una temática, es decir, cuantos comentarios están relacionados con un mismo tema por ejemplo. 

Esto se puede calcular utilizando la matriz de temas de un documento y sumando las contribuciones individuales a este a lo largo de todos los documentos del data frame.

In [10]:
W_text_matrix.sum(axis=0)/W_text_matrix.sum()*100.0

array([18.15077823, 11.61180167,  5.36082748,  9.99092406,  8.43365604,
       10.84861004,  7.45875981, 12.96782206,  6.9400853 ,  8.23673532])

Este resultado indica que hay temas de mayor y menor peso pero no hay grandes diferencias en los porcentajes, indicando una supuesta buena calidad al tener una distribución bastante similar.

### Creación de Modelos temáticos utilizando SVD
Otro algoritmo utilizado para el modelo de temas es el basado en la *descomposición de valores singular (SVD)* 

Se trata de una técnica de factorización matricial utilizada para reducir la dimensionalidad de los datos y extraer patrones semánticos latentes. En este método, la matriz de datos es descompuesta en tres matrices: *U*, que representa la relación entre documentos y conceptos latentes; *Σ*, que contiene los valores singulares que indican la importancia de cada concepto; y *V^T*, que relaciona términos con estos conceptos.

En el procesamiento de lenguaje natural habitualmente se utiliza una variante de este algoritmo denominada *Truncated SVD* que reduce considerablemente el costo computacional.

In [11]:
from sklearn.decomposition import TruncatedSVD

svd_text_model = TruncatedSVD(n_components = 10, random_state=42)
W_svd_text_matrix = svd_text_model.fit_transform(tfidf_text_vectors)
H_svd_text_matrix = svd_text_model.components_

La función antes definida para evaluar y mostrar por pantalla el modelo puede ser reutilizado para este método.

In [12]:
display_topics(svd_text_model, tfidf_text_vectorizer.get_feature_names_out())


Topic 00
  stale (15.99)
  days (15.34)
  activity (7.94)
  label (7.94)
  comment (7.85)

Topic 01
  zigbee2mqtt (1.53)
  issue (1.20)
  _url_ (1.04)
  config (0.94)
  error (0.88)

Topic 02
  thanks (52.50)
  issue (8.42)
  _url_ (6.00)
  worked (3.65)
  working (2.58)

Topic 03
  issue (17.00)
  _url_ (13.62)
  duplicate (2.07)
  fixed (2.04)
  related (1.32)

Topic 04
  _url_ (65.02)
  duplicate (10.46)
  stale (6.17)
  days (5.29)
  related (5.24)

Topic 05
  _url_ (8.70)
  issue (7.85)
  error (7.07)
  herdsman (6.39)
  zigbee (6.35)

Topic 06
  problem (1525.35)
  solution (208.44)
  solved (189.74)
  version (145.35)
  zigbee (143.70)

Topic 07
  update (50.30)
  version (49.80)
  edge (36.24)
  latest (18.68)
  fixed (15.63)

Topic 08
  fixed (119.10)
  dev (55.34)
  edge (51.06)
  branch (39.53)
  z2m (32.51)

Topic 09
  addon (6.22)
  zigbee2mqtt (5.33)
  fixed (4.58)
  update (4.50)
  config (3.81)


In [13]:
svd_text_model.singular_values_

array([18.83292308,  8.18662956,  6.33037565,  6.15879161,  5.60076025,
        5.34577855,  4.83188556,  4.51548091,  4.19867845,  4.0088322 ])

Al igual que con NMF, los valores singulares obtenidos representan la importancia relativa de cada componente latente en la estructura del texto. Un valor más alto indica que el componente correspondiente explica una mayor variabilidad en los datos, lo que sugiere que los temas asociados a esos componentes son más representativos en el corpus. A medida que los valores disminuyen, los temas capturados tienen menor influencia en la distribución global de términos y documentos. Esto permite reducir la dimensionalidad del texto, reteniendo solo las estructuras más significativas para el análisis.

Ambos métodos vistos utilizan métodos algebraicos usando como base la matriz de términos para la descomposición de temas. A partir de este punto, se verán modelos probabilísticos cuya popularidad ha aumentado en los últimos años.

## Asignación latente de Dirichlet (LDA)
LDA es el método de modelado de tópicos más popular en los últimos tiempos gracias a su capacidad de adaptación en diferentes escenarios.

Su funcionamiento consiste en leer cada documento como una mezcla de diferentes temas que a su vez son una mezcla de palabras. Para mantener una cantidad reducida de estos hace uso de la distribución Dirichlet que asegura que cada documento esté compuesto por unos pocos tópicos que al mismo tiempo están compuestos por una cantidad reducida de palabras.

### Creación de Modelos temáticos utilizando LDA
Para este tipo de modelos también se utilizará la librería *scikit-learn*. Debido a que se hace uso de un método probabilístico, la duración del proceso es mayor en comparación con NMF y SVD.

In [14]:
from sklearn.feature_extraction.text import CountVectorizer

count_text_vectorizer = CountVectorizer(stop_words=list(stopwords), min_df=5, max_df=0.7)
count_text_vectors = count_text_vectorizer.fit_transform(df['normalized_text'])
count_text_vectors.shape

(2678, 1721)

In [15]:
from sklearn.decomposition import LatentDirichletAllocation

lda_text_model = LatentDirichletAllocation(n_components = 10, random_state=42)
W_lda_text_matrix = lda_text_model.fit_transform(count_text_vectors)
H_lda_text_matrix = lda_text_model.components_

En este caso también se puede utilizar la función antes definida para la evaluación y muestra de los resultados:

In [16]:
display_topics(lda_text_model, count_text_vectorizer.get_feature_names_out())


Topic 00
  add (2.30)
  version (1.94)
  zigbee2mqtt (1.93)
  addon (1.74)
  issue (1.71)

Topic 01
  false (4.80)
  null (3.89)
  failed (2.97)
  zigbee2mqtt (2.81)
  state (2.71)

Topic 02
  ezsp (5.00)
  zigbee (4.44)
  01 (4.30)
  adapter (3.80)
  herdsman (3.63)

Topic 03
  days (13.49)
  stale (12.90)
  issue (7.89)
  open (7.75)
  remove (7.53)

Topic 04
  zigbee (4.76)
  herdsman (4.03)
  zigbee2mqtt (3.47)
  error (3.40)
  adapter (2.97)

Topic 05
  config (3.35)
  mqtt (2.77)
  file (2.59)
  port (2.54)
  zigbee2mqtt (2.30)

Topic 06
  info (5.37)
  root (3.65)
  zigbee2mqtt (3.59)
  22 (2.16)
  11 (1.39)

Topic 07
  z2m (2.32)
  _url_ (2.00)
  problem (1.93)
  issue (1.72)
  device (1.36)

Topic 08
  2022 (7.85)
  08 (6.97)
  10 (5.63)
  zigbee2mqtt (5.55)
  30 (5.29)

Topic 09
  mainthread (7.16)
  10 (6.33)
  zigbee2mqtt (4.70)
  option (4.37)
  warning (4.32)


In [17]:
W_lda_text_matrix.sum(axis=0)/W_lda_text_matrix.sum()*100.0

array([32.30908695,  3.43165214,  3.47439399, 16.02876733,  5.04281378,
        6.09383153,  5.47115736, 22.13451431,  2.4828182 ,  3.53096439])

A simple vista se aprecia que LDA ha generado una estructura de temas bastante diferente en comparación con las anteriores. Además, en los resultados hay un porcentaje que destaca por ser muy grande en comparación con los demás, indicando así que la calidad del modelo puede mejorarse pues la distribución no es muy equivalente. Esto se podría conseguir variando el número de documentos elegidos para la ejecución hasta conseguir una distribución con menos diferencia entre los resultados que se obtengan.

#### Visualización de los resultados de LDA
La librería pyLDAvis es útil para la visualización de los resultados del modelado LDA tomando directamente los resultados obtenidos previamente.

In [24]:
import pyLDAvis
import pyLDAvis.lda_model

lda_display = pyLDAvis.lda_model.prepare(
    lda_text_model,  # Modelo LDA entrenado
    count_text_vectors,  # Matriz documento-término
    count_text_vectorizer,  # Vectorizador (para obtener vocabulario)
    sort_topics=False
)

pyLDAvis.display(lda_display)

In [26]:
lda_tsne_display = pyLDAvis.lda_model.prepare(lda_text_model, count_text_vectors, count_text_vectorizer, sort_topics=False, mds='tsne')
pyLDAvis.display(lda_tsne_display)

La visualización generada por pyLDAvis ofrece una representación gráfica de los temas extraídos de un conjunto de documentos. En ella, cada tema se representa como un círculo cuyo tamaño indica su prominencia en el conjunto de datos, mientras que la posición refleja su relación con otros temas. Las palabras más frecuentes y representativas de cada tema se muestran junto al círculo correspondiente, lo que permite interpretar el contenido de los temas. Esta visualización facilita la comprensión de las principales categorías presentes en los textos, ayuda a identificar patrones y relaciones entre los temas, y es útil para asignar documentos a sus temas más relevantes o explorar soluciones a problemas comunes en grandes volúmenes de texto.

### Utilización de nubes de palabras para mostrar y comparar modelos (Pág 222)